# 라이브러리 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install koreanize-matplotlib # 한국어 표기를 위한 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import koreanize_matplotlib
%matplotlib inline

# import graphviz #나무구조 시각화
from sklearn.tree import export_graphviz #나무구조 생성 및 저장
from sklearn.model_selection import train_test_split #데이터 분할
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance # 변수 중요도 확인 : F1 점수 기준
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV #최적모델, 파라미터 탐색
from sklearn.preprocessing import StandardScaler
from statsmodels.api import Logit
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC# 분류모델 평가함수
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestRegressor

from statsmodels.formula.api import ols # 회귀분석
from statsmodels.api import qqplot, add_constant # qqplot, 회귀분석의 상수항 추가
import statsmodels.formula.api as smf # 선형 모델 함수

from statsmodels.tools.eval_measures import rmse # 평가함수
# 주성분 분석
from sklearn.decomposition import PCA
# 회귀분석
import statsmodels.api as sm


In [4]:
pd.options.display.precision =2 # 데이터 소수점 지정

# Customer 데이터 불러오기

In [5]:
df_raw = pd.read_csv("/content/drive/MyDrive/telecom/telco_customer.csv",encoding = 'euc-kr')
df_raw.head()

,CustomerId,Gender,Age,AgeGroup,Married,Dependents,noDependents,Referrals,noReferrals,PaperlessBilling,...,EndDateTmp,ChurnLabel,ChurnScore,ChurnCategory,ChurnReason,CustomerLTV,SatisScore,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge
0,C-10001,여성,37,30대,Yes,No,0,Yes,2,Yes,...,2022-01-01,No,65,NaN,NaN,2205,3,0,47910,431100
1,C-10002,남성,46,40대,No,No,0,No,0,No,...,2022-01-01,No,66,NaN,NaN,5414,5,11300,12090,108710
2,C-10003,남성,50,50대,No,No,0,No,0,Yes,...,2021-02-01,Yes,71,Competitor,Price too high,4479,1,0,38080,152100
3,C-10004,남성,78,70대,Yes,No,0,Yes,1,Yes,...,2021-07-01,Yes,99,Dissatisfaction,Product dissatisfaction,3714,1,0,31410,408720
4,C-10005,여성,75,70대,Yes,No,0,Yes,3,Yes,...,2021-10-01,Yes,68,Dissatisfaction,Network reliability,3464,1,0,8360,24970


In [6]:
df_raw.shape

(6863, 28)

In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6863 entries, 0 to 6862
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CustomerId            6863 non-null   object
 1   Gender                6863 non-null   object
 2   Age                   6863 non-null   int64 
 3   AgeGroup              6863 non-null   object
 4   Married               6863 non-null   object
 5   Dependents            6863 non-null   object
 6   noDependents          6863 non-null   int64 
 7   Referrals             6863 non-null   object
 8   noReferrals           6863 non-null   int64 
 9   PaperlessBilling      6863 non-null   object
 10  PaymentMethod         6863 non-null   object
 11  OnlineSecurity        6863 non-null   object
 12  OnlineBackup          6863 non-null   object
 13  TechSupport           6863 non-null   object
 14  UnlimitedData         6863 non-null   object
 15  AvgDownloadGB         6863 non-null   

In [8]:
df_raw.describe()

,Age,noDependents,noReferrals,AvgDownloadGB,ChurnScore,CustomerLTV,SatisScore,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge
count,6863.00,6863.00,6863.00,6863.00,6863.00,6863.00,6863.00,6863.00,6863.00,6.86e+03
mean,46.61,0.41,1.99,20.51,58.13,4412.44,3.28,7916.42,25966.32,8.67e+05
std,16.58,0.88,3.02,20.42,21.41,1180.38,1.19,28684.75,17477.41,9.60e+05
min,20.00,0.00,0.00,0.00,5.00,2003.00,1.00,0.00,0.00,0.00e+00
25%,32.00,0.00,0.00,3.00,40.00,3487.00,3.00,0.00,10400.00,9.44e+04
50%,46.00,0.00,0.00,17.00,60.00,4538.00,3.00,0.00,25990.00,4.86e+05
75%,60.00,0.00,3.00,27.00,75.00,5388.00,4.00,0.00,41250.00,1.38e+06
max,80.00,4.00,11.00,85.00,100.00,6500.00,5.00,169500.00,56500.00,4.03e+06


In [9]:
df_raw.describe(include = "object")

,CustomerId,Gender,AgeGroup,Married,Dependents,Referrals,PaperlessBilling,PaymentMethod,OnlineSecurity,OnlineBackup,TechSupport,UnlimitedData,StartDate,EndDate,EndDateTmp,ChurnLabel,ChurnCategory,ChurnReason
count,6863,6863,6863,6863,6863,6863,6863,6863,6863,6863,6863,6863,6863,1702,6863,6863,1702,1702
unique,6863,2,7,2,2,2,2,3,2,2,2,2,82,12,13,2,5,20
top,C-10001,남성,20대,No,No,No,Yes,계좌이체,No,No,No,Yes,2015-01-01,2021-10-01,2022-01-01,No,Competitor,Attitude of support person
freq,1,3472,1344,3896,5410,3682,4057,3799,4855,4457,4828,4615,356,235,5161,5161,762,176


In [10]:
df_raw.isnull().sum()

CustomerId                 0
Gender                     0
Age                        0
AgeGroup                   0
Married                    0
Dependents                 0
noDependents               0
Referrals                  0
noReferrals                0
PaperlessBilling           0
PaymentMethod              0
OnlineSecurity             0
OnlineBackup               0
TechSupport                0
UnlimitedData              0
AvgDownloadGB              0
StartDate                  0
EndDate                 5161
EndDateTmp                 0
ChurnLabel                 0
ChurnScore                 0
ChurnCategory           5161
ChurnReason             5161
CustomerLTV                0
SatisScore                 0
TotalExtraDataCharge       0
AvgRoamCharge              0
TotalRoamCharge            0
dtype: int64

In [11]:
df_raw = df_raw.drop(['EndDate','ChurnCategory','ChurnReason','EndDateTmp','StartDate'], axis = 1)

In [12]:
df_raw_dummy = pd.get_dummies(df_raw)
df_raw_dummy.head()

,Age,noDependents,noReferrals,AvgDownloadGB,ChurnScore,CustomerLTV,SatisScore,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,...,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,TechSupport_No,TechSupport_Yes,UnlimitedData_No,UnlimitedData_Yes,ChurnLabel_No,ChurnLabel_Yes
0,37,0,2,16,65,2205,3,0,47910,431100,...,1,0,0,1,0,1,0,1,1,0
1,46,0,0,10,66,5414,5,11300,12090,108710,...,1,0,1,0,1,0,1,0,1,0
2,50,0,0,30,71,4479,1,0,38080,152100,...,1,0,1,0,1,0,0,1,0,1
3,78,0,1,4,99,3714,1,0,31410,408720,...,1,0,0,1,1,0,0,1,0,1
4,75,0,3,11,68,3464,1,0,8360,24970,...,1,0,1,0,0,1,0,1,0,1


In [13]:
# 설명변수, 목표변수 데이터 구분
df_raw_x = df_raw_dummy.drop(['ChurnLabel_Yes','ChurnLabel_No'],axis = 1, inplace = False)
df_raw_y = df_raw_dummy['ChurnLabel_Yes']
# 데이터 분할 Train_test_split'
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, df_raw_y, test_size = 0.3)

print(f'분할 전 설명변수 데이터: {df_raw_x.shape}')
print(f'''분할 후 설명변수 데이터:
Train {df_train_x.shape}
Test {df_test_x.shape}''')

분할 전 설명변수 데이터: (6863, 6901)
분할 후 설명변수 데이터: 
Train (4804, 6901)
Test (2059, 6901)


In [14]:
print(df_raw_dummy.value_counts(['ChurnLabel_Yes']))
print('ChurnLabel_yes = 1의 비율', df_raw_dummy.value_counts(df_raw_dummy['ChurnLabel_Yes']==1)/len(df_raw_dummy))

# oversampling 할 필요없음


ChurnLabel_Yes
0                 5161
1                 1702
dtype: int64
ChurnLabel_yes = 1의 비율 ChurnLabel_Yes
False    0.75
True     0.25
dtype: float64


In [15]:
# 의사결정나무 모델 생성
tree_uncustomized = DecisionTreeClassifier()
tree_uncustomized.fit(df_train_x, df_train_y)
# train 데이터 정확도
print('Accuracy on training set:{:.3f}'.format(tree_uncustomized.score(df_train_x, df_train_y)))
# test 데이터 정확도
print("Accuracy on test set:{:.3f}".format(tree_uncustomized.score(df_test_x, df_test_y)))

## 모델 정확도는 train 100%, test 97%로 과대적합이나 지금은
## 변수 중요도를 알아보기 위한 단계이므로 위 값을 건더 뛰겠음


Accuracy on training set:1.000
Accuracy on test set:0.977


In [16]:
tree_uncustomized

DecisionTreeClassifier()

In [17]:
v_feature_name = df_train_x.columns
v_feature_name

Index(['Age', 'noDependents', 'noReferrals', 'AvgDownloadGB', 'ChurnScore',
       'CustomerLTV', 'SatisScore', 'TotalExtraDataCharge', 'AvgRoamCharge',
       'TotalRoamCharge',
       ...
       'PaymentMethod_신용카드', 'PaymentMethod_이체/메일확인', 'OnlineSecurity_No',
       'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_Yes',
       'TechSupport_No', 'TechSupport_Yes', 'UnlimitedData_No',
       'UnlimitedData_Yes'],
      dtype='object', length=6901)

In [18]:
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = tree_uncustomized.feature_importances_


In [19]:
df_importance

,Feature,Importance
0,Age,1.66e-03
1,noDependents,3.00e-03
2,noReferrals,8.70e-03
3,AvgDownloadGB,1.62e-02
4,ChurnScore,1.60e-01
...,...,...
6896,OnlineBackup_Yes,0.00e+00
6897,TechSupport_No,0.00e+00
6898,TechSupport_Yes,0.00e+00
6899,UnlimitedData_No,7.50e-04


In [20]:


df_importance.sort_values('Importance', ascending = False, inplace = True)
coordinates = range(len(df_importance['Importance']))

df_importance

,Feature,Importance
6,SatisScore,0.73
4,ChurnScore,0.16
6894,OnlineSecurity_Yes,0.02
3,AvgDownloadGB,0.02
9,TotalRoamCharge,0.01
...,...,...
2329,CustomerId_C-12378,0.00
2328,CustomerId_C-12377,0.00
2327,CustomerId_C-12376,0.00
2326,CustomerId_C-12375,0.00


In [21]:
# 랜던 포레스트 모델 생성 : RandomForestclassifier
rf_uncustomized = RandomForestClassifier()
rf_uncustomized.fit(df_train_x, df_train_y)

# 모델 평가를 위한 것이 아니라 feature importance를 통해 vital few를 도출하기 위한 것이므로 그냥 넘어가겠음



RandomForestClassifier()

In [22]:
# 변수명
v_feature_name = df_train_x.columns
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = rf_uncustomized.feature_importances_
df_importance.sort_values('Importance', ascending = False,inplace = True)


In [23]:
df_importance

,Feature,Importance
6,SatisScore,0.24
4,ChurnScore,0.19
9,TotalRoamCharge,0.04
3,AvgDownloadGB,0.04
5,CustomerLTV,0.03
...,...,...
4700,CustomerId_C-14816,0.00
1756,CustomerId_C-11789,0.00
4704,CustomerId_C-14820,0.00
1754,CustomerId_C-11787,0.00


In [24]:
# 그라디언트 부스팅 모델 생성
gb_uncustomized = GradientBoostingClassifier()
gb_uncustomized.fit(df_train_x, df_train_y)


GradientBoostingClassifier()

In [25]:
gb_uncustomized

GradientBoostingClassifier()

In [26]:
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = gb_uncustomized.feature_importances_
df_importance.sort_values("Importance",ascending = False, inplace = True)

In [27]:
df_importance

,Feature,Importance
6,SatisScore,7.91e-01
4,ChurnScore,1.80e-01
6893,OnlineSecurity_No,3.77e-03
2,noReferrals,2.14e-03
3,AvgDownloadGB,1.45e-03
...,...,...
2323,CustomerId_C-12372,0.00e+00
2322,CustomerId_C-12371,0.00e+00
2321,CustomerId_C-12370,0.00e+00
6900,UnlimitedData_Yes,0.00e+00


In [28]:
df_train_x

,Age,noDependents,noReferrals,AvgDownloadGB,ChurnScore,CustomerLTV,SatisScore,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,...,PaymentMethod_신용카드,PaymentMethod_이체/메일확인,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,TechSupport_No,TechSupport_Yes,UnlimitedData_No,UnlimitedData_Yes
1265,38,1,1,24,36,4879,5,0,53900,3125580,...,0,0,1,0,0,1,1,0,0,1
13,32,0,0,16,53,5513,3,0,2600,183060,...,0,0,0,1,0,1,1,0,0,1
3481,61,1,8,21,73,3147,3,0,50400,201820,...,0,0,0,1,1,0,1,0,0,1
6431,20,0,0,59,27,2496,3,0,41580,1787100,...,1,0,0,1,0,1,0,1,0,1
1257,71,0,0,23,70,4932,4,0,0,0,...,0,0,1,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,39,0,0,13,83,3865,1,0,11300,45090,...,1,0,1,0,1,0,1,0,0,1
2103,33,0,0,20,46,6183,3,0,48700,3217340,...,0,0,0,1,0,1,0,1,0,1
1833,73,0,1,9,76,4989,1,0,27350,27350,...,0,0,0,1,1,0,1,0,0,1
2274,38,0,0,8,25,5674,5,0,15710,188260,...,0,0,1,0,1,0,1,0,0,1


In [29]:
df_train_x = df_train_x.drop('SatisScore',axis = 1)

In [30]:
df_train_x

,Age,noDependents,noReferrals,AvgDownloadGB,ChurnScore,CustomerLTV,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,CustomerId_C-10001,...,PaymentMethod_신용카드,PaymentMethod_이체/메일확인,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,TechSupport_No,TechSupport_Yes,UnlimitedData_No,UnlimitedData_Yes
1265,38,1,1,24,36,4879,0,53900,3125580,0,...,0,0,1,0,0,1,1,0,0,1
13,32,0,0,16,53,5513,0,2600,183060,0,...,0,0,0,1,0,1,1,0,0,1
3481,61,1,8,21,73,3147,0,50400,201820,0,...,0,0,0,1,1,0,1,0,0,1
6431,20,0,0,59,27,2496,0,41580,1787100,0,...,1,0,0,1,0,1,0,1,0,1
1257,71,0,0,23,70,4932,0,0,0,0,...,0,0,1,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,39,0,0,13,83,3865,0,11300,45090,0,...,1,0,1,0,1,0,1,0,0,1
2103,33,0,0,20,46,6183,0,48700,3217340,0,...,0,0,0,1,0,1,0,1,0,1
1833,73,0,1,9,76,4989,0,27350,27350,0,...,0,0,0,1,1,0,1,0,0,1
2274,38,0,0,8,25,5674,0,15710,188260,0,...,0,0,1,0,1,0,1,0,0,1


In [32]:
# 의사결정나무 모델 생성
tree_uncustomized = DecisionTreeClassifier()
tree_uncustomized.fit(df_train_x, df_train_y)

DecisionTreeClassifier()

In [33]:
tree_uncustomized

DecisionTreeClassifier()

In [34]:
v_feature_name = df_train_x.columns
v_feature_name

Index(['Age', 'noDependents', 'noReferrals', 'AvgDownloadGB', 'ChurnScore',
       'CustomerLTV', 'TotalExtraDataCharge', 'AvgRoamCharge',
       'TotalRoamCharge', 'CustomerId_C-10001',
       ...
       'PaymentMethod_신용카드', 'PaymentMethod_이체/메일확인', 'OnlineSecurity_No',
       'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_Yes',
       'TechSupport_No', 'TechSupport_Yes', 'UnlimitedData_No',
       'UnlimitedData_Yes'],
      dtype='object', length=6900)

In [35]:
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = tree_uncustomized.feature_importances_


In [37]:
df_importance.sort_values('Importance', ascending = False, inplace = True)
coordinates = range(len(df_importance['Importance']))

df_importance

,Feature,Importance
4,ChurnScore,0.64
3,AvgDownloadGB,0.05
2,noReferrals,0.04
8,TotalRoamCharge,0.03
7,AvgRoamCharge,0.03
...,...,...
2324,CustomerId_C-12374,0.00
2323,CustomerId_C-12373,0.00
2322,CustomerId_C-12372,0.00
2321,CustomerId_C-12371,0.00


In [38]:
# 랜던 포레스트 모델 생성 : RandomForestclassifier
rf_uncustomized = RandomForestClassifier()
rf_uncustomized.fit(df_train_x, df_train_y)

# 모델 평가를 위한 것이 아니라 feature importance를 통해 vital few를 도출하기 위한 것이므로 그냥 넘어가겠음



RandomForestClassifier()

In [39]:
# 변수명
v_feature_name = df_train_x.columns
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = rf_uncustomized.feature_importances_
df_importance.sort_values('Importance', ascending = False,inplace = True)


In [40]:
df_importance

,Feature,Importance
4,ChurnScore,0.25
8,TotalRoamCharge,0.05
3,AvgDownloadGB,0.05
5,CustomerLTV,0.04
7,AvgRoamCharge,0.04
...,...,...
4251,CustomerId_C-14360,0.00
5428,CustomerId_C-15568,0.00
1422,CustomerId_C-11455,0.00
2451,CustomerId_C-12503,0.00


In [41]:
# 그라디언트 부스팅 모델 생성
gb_uncustomized = GradientBoostingClassifier()
gb_uncustomized.fit(df_train_x, df_train_y)


GradientBoostingClassifier()

In [42]:
gb_uncustomized

GradientBoostingClassifier()

In [43]:
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = gb_uncustomized.feature_importances_
df_importance.sort_values("Importance",ascending = False, inplace = True)

In [44]:
df_importance

,Feature,Importance
4,ChurnScore,0.84
2,noReferrals,0.04
3,AvgDownloadGB,0.03
8,TotalRoamCharge,0.03
6890,PaymentMethod_신용카드,0.01
...,...,...
2322,CustomerId_C-12372,0.00
2321,CustomerId_C-12371,0.00
2320,CustomerId_C-12370,0.00
2319,CustomerId_C-12369,0.00


In [45]:
df_train_x = df_train_x.drop('ChurnScore',axis = 1)

In [46]:
df_train_x

,Age,noDependents,noReferrals,AvgDownloadGB,CustomerLTV,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,CustomerId_C-10001,CustomerId_C-10002,...,PaymentMethod_신용카드,PaymentMethod_이체/메일확인,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,TechSupport_No,TechSupport_Yes,UnlimitedData_No,UnlimitedData_Yes
1265,38,1,1,24,4879,0,53900,3125580,0,0,...,0,0,1,0,0,1,1,0,0,1
13,32,0,0,16,5513,0,2600,183060,0,0,...,0,0,0,1,0,1,1,0,0,1
3481,61,1,8,21,3147,0,50400,201820,0,0,...,0,0,0,1,1,0,1,0,0,1
6431,20,0,0,59,2496,0,41580,1787100,0,0,...,1,0,0,1,0,1,0,1,0,1
1257,71,0,0,23,4932,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,39,0,0,13,3865,0,11300,45090,0,0,...,1,0,1,0,1,0,1,0,0,1
2103,33,0,0,20,6183,0,48700,3217340,0,0,...,0,0,0,1,0,1,0,1,0,1
1833,73,0,1,9,4989,0,27350,27350,0,0,...,0,0,0,1,1,0,1,0,0,1
2274,38,0,0,8,5674,0,15710,188260,0,0,...,0,0,1,0,1,0,1,0,0,1


In [47]:
# 의사결정나무 모델 생성
tree_uncustomized = DecisionTreeClassifier()
tree_uncustomized.fit(df_train_x, df_train_y)

DecisionTreeClassifier()

In [48]:
tree_uncustomized

DecisionTreeClassifier()

In [49]:
v_feature_name = df_train_x.columns
v_feature_name

Index(['Age', 'noDependents', 'noReferrals', 'AvgDownloadGB', 'CustomerLTV',
       'TotalExtraDataCharge', 'AvgRoamCharge', 'TotalRoamCharge',
       'CustomerId_C-10001', 'CustomerId_C-10002',
       ...
       'PaymentMethod_신용카드', 'PaymentMethod_이체/메일확인', 'OnlineSecurity_No',
       'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_Yes',
       'TechSupport_No', 'TechSupport_Yes', 'UnlimitedData_No',
       'UnlimitedData_Yes'],
      dtype='object', length=6899)

In [50]:
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = tree_uncustomized.feature_importances_


In [51]:
df_importance.sort_values('Importance', ascending = False, inplace = True)
coordinates = range(len(df_importance['Importance']))

df_importance

,Feature,Importance
3,AvgDownloadGB,0.12
2,noReferrals,0.10
7,TotalRoamCharge,0.09
4,CustomerLTV,0.08
6,AvgRoamCharge,0.07
...,...,...
2364,CustomerId_C-12416,0.00
2363,CustomerId_C-12415,0.00
2362,CustomerId_C-12414,0.00
2361,CustomerId_C-12413,0.00


In [52]:
# 랜던 포레스트 모델 생성 : RandomForestclassifier
rf_uncustomized = RandomForestClassifier()
rf_uncustomized.fit(df_train_x, df_train_y)

# 모델 평가를 위한 것이 아니라 feature importance를 통해 vital few를 도출하기 위한 것이므로 그냥 넘어가겠음



RandomForestClassifier()

In [53]:
# 변수명
v_feature_name = df_train_x.columns
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = rf_uncustomized.feature_importances_
df_importance.sort_values('Importance', ascending = False,inplace = True)


In [54]:
df_importance

,Feature,Importance
7,TotalRoamCharge,0.07
4,CustomerLTV,0.06
3,AvgDownloadGB,0.06
6,AvgRoamCharge,0.06
0,Age,0.06
...,...,...
2523,CustomerId_C-12578,0.00
2521,CustomerId_C-12576,0.00
2518,CustomerId_C-12573,0.00
5200,CustomerId_C-15334,0.00


In [55]:
# 그라디언트 부스팅 모델 생성
gb_uncustomized = GradientBoostingClassifier()
gb_uncustomized.fit(df_train_x, df_train_y)


GradientBoostingClassifier()

In [56]:
gb_uncustomized

GradientBoostingClassifier()

In [57]:
df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = gb_uncustomized.feature_importances_
df_importance.sort_values("Importance",ascending = False, inplace = True)

In [58]:
df_importance

,Feature,Importance
2,noReferrals,0.26
3,AvgDownloadGB,0.14
7,TotalRoamCharge,0.09
6889,PaymentMethod_신용카드,0.06
6,AvgRoamCharge,0.05
...,...,...
2328,CustomerId_C-12379,0.00
2327,CustomerId_C-12378,0.00
2326,CustomerId_C-12377,0.00
2324,CustomerId_C-12375,0.00


In [59]:
# 고객만족도와 이탈점수를 제외하니
# 다운로드데이터, 총로밍요금, 추천횟수가 중요한 변수(Vitalfew)로 나왔음
